## MonReader - part 4

----

### Text-to-Speech Generation from OCR Transcriptions (Audiobook-Style Rendering)

**Objective.**  
Transform the page-level OCR transcriptions produced in **MonReader – Part 3** into **spoken audio**, enabling end-to-end document accessibility and audiobook-style consumption.  
This part evaluates modern **Text-to-Speech (TTS)** models on long-form literary text, focusing on intelligibility, prosody, multilingual robustness, and generation efficiency.

We use the same two sources as previous parts:
- *The Chamber* — John Grisham *(English)*
- *A onda que se ergueu no mar* — Ruy Castro *(Portuguese)*

All text input is sourced from the verified per-page `.txt` outputs generated by the best-performing OCR model (**Qwen2.5-VL**), ensuring a clean and stable foundation for speech synthesis.

**Why this experiment.**  
While OCR converts visual documents into text, **Text-to-Speech closes the accessibility loop**, enabling hands-free reading, audio archiving, and downstream multimodal applications helping the blind, the researchers and for everyone else in need for fully automatic, highly fast and high-quality document scanning in bulk.

Long-form literary TTS presents challenges beyond short prompts: paragraph continuity, punctuation-aware prosody, proper-noun pronunciation, multilingual phonemes, and stability over extended generation windows. This experiment measures how well current TTS systems handle these constraints in a realistic, book-scale setting.

**Proposed Pipeline Overview**

1. **I – Data Preparation**  
   Normalize and structure OCR text for speech synthesis: page ordering, light TTS-oriented text normalization (hyphen joins, punctuation handling), language tagging, and chunking to respect model limits.

2. **J – TTS Model Benchmarking**  
   Evaluate three leading Text-to-Speech models on representative excerpts from both books.  
   Comparison dimensions include generation speed, pronunciation accuracy, prosody, multilingual handling (English vs. Portuguese), and long-form stability.

3. **K – Full Audio Generation**  
   Using the selected TTS model and fixed configuration, generate per-page audio files for both books.  
   Outputs include structured audio manifests and reproducible file layouts suitable for later merging into chapter- or book-level recordings.

> This part prioritizes **reproducibility and minimal intervention**: the OCR pipeline remains unchanged, and only lightweight, TTS-specific normalization is applied to the extracted text.


----


#### Imports and Environment

In [1]:
from pathlib import Path
import re
import json
import time
import math
import unicodedata
import pandas as pd
from typing import List, Dict, Iterable

In [2]:
BASE = Path.cwd()
WORK_DIR = BASE / "work"

# OCR outputs from Part 3
STEPH_DIR = WORK_DIR / "stepH_qwen2p5vl_full"

# Part 4 outputs
STEP4_DIR = WORK_DIR / "step4_tts"
STEP4_DIR.mkdir(parents=True, exist_ok=True)


----

### Step I — Data Preparation for Text-to-Speech

**Goal.**  
Transform OCR-derived page outputs into **clean, continuous, speech-ready text** while preserving semantic content and correct reading order.

This step performs **strictly minimal, TTS-oriented processing**, with no OCR correction or semantic rewriting. The focus is on structural consistency and speech robustness.

**Processing stages in this step:**

1) inspection of per-page OCR JSON structure
2) explicit verification of page ordering
3) extraction and concatenation of page-level text
4) removal of OCR and formatting artifacts
5) repair of hyphenated and line-wrapped words
6) generation of canonical continuous prose
7) sentence-aware chunking to respect TTS model limits


No linguistic enhancement, paraphrasing, or content modification is applied. The output of this step serves as the **single source of truth** for all subsequent Text-to-Speech benchmarking and audio generation.




#### Step I.1 - Inspection of per-page OCR JSON structure

Each page has one JSON like:

```pgsql
work/
└─ stepH_qwen2p5vl_full/
   └─ <BOOK_ID>/
      └─ json/
         └─ pagXX.json
```


In [12]:
from pprint import pprint

In [ ]:
# print the JSON structure of the page12 from the first book
BOOK_ID = "A_onda_que_se_ergueu_no_mar-Ruy_Castro"
PAGE = "pag12"

json_path = Path("work/stepH_qwen2p5vl_full") / BOOK_ID / "json" / f"{PAGE}.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

pprint(data)

{'image': 'pag12.JPEG',
 'image_path': 'e:\\Devs\\pyEnv-1\\Apziva\\MonReader\\data\\books\\A_onda_que_se_ergueu_no_mar-Ruy_Castro\\images\\pag12.JPEG',
 'latency_s': 870.2552971839905,
 'model': 'qwen2.5vl:latest',
 'options': {'num_predict': 1536,
             'repeat_penalty': 1.25,
             'stop': ['}\n', '}\r\n', '}'],
             'temperature': 0,
             'top_p': 1},
 'parsed': {'degenerate': False,
            'json_objs': 0,
            'language': 'guess',
            'lines': ['{',
                      '  "language": "por",',
                      '  "lines": [',
                      '    "A trilha",',
                      '    "sonora de um",',
                      '    "país ideal",',
                      '    "",',
                      '    "Olha que coisa mais linda: as garotas de '
                      'Ipanema-1961",',
                      '    "tomavam cuba-libre, dirigiam Kharman-Ghias e '
                      'voavam",',
                      '   

#### Step I.2 - Explicit verification of page ordering

Page filenames like "pag2.JPEG" vs "pag10.JPEG" will sort incorrectly with plain string sorting.
We define an explicit "natural sort" that extracts the numeric page id and sorts by it, the functions were included in the `MonReader_tools.py` library.


In [16]:
import importlib
import MonReader_tools

importlib.reload(MonReader_tools)

from MonReader_tools import (
    page_number_from_name, list_pages_sorted, verify_page_order
)

In [37]:
# Run verification for both books (Step H JSON outputs)

BOOKS = [
    ("The_Chamber-John_Grisham", "eng"),
    ("A_onda_que_se_ergueu_no_mar-Ruy_Castro", "por"),
]

dfs = {}
for b,_ in BOOKS:
    dfs[b] = verify_page_order(b, which="json", preview=50)
    display(dfs[b].head(20))


>>> The_Chamber-John_Grisham (json)
Directory: e:\Devs\pyEnv-1\Apziva\MonReader\work\stepH_qwen2p5vl_full\The_Chamber-John_Grisham\json
Order preview:
 - pag0.json
 - pag2.json
 - pag4.json
 - pag6.json
 - pag8.json
 - pag10.json
 - pag12.json
 - pag14.json
 - pag16.json
 - pag18.json
 - pag20.json
 - pag22.json


,name,page_n
0,pag0.json,0
1,pag2.json,2
2,pag4.json,4
3,pag6.json,6
4,pag8.json,8
5,pag10.json,10
6,pag12.json,12
7,pag14.json,14
8,pag16.json,16
9,pag18.json,18



>>> A_onda_que_se_ergueu_no_mar-Ruy_Castro (json)
Directory: e:\Devs\pyEnv-1\Apziva\MonReader\work\stepH_qwen2p5vl_full\A_onda_que_se_ergueu_no_mar-Ruy_Castro\json
Order preview:
 - pag12.json
 - pag16.json
 - pag18.json
 - pag20.json
 - pag22.json
 - pag24.json
 - pag26.json
 - pag28.json
 - pag32.json
 - pag36.json
 - pag40.json
 - pag44.json


,name,page_n
0,pag12.json,12
1,pag16.json,16
2,pag18.json,18
3,pag20.json,20
4,pag22.json,22
5,pag24.json,24
6,pag26.json,26
7,pag28.json,28
8,pag32.json,32
9,pag36.json,36


### Step I.3 - extraction and concatenation of page-level text

Goal:
- Read all per-page JSON artifacts in *correct numeric order*
- Extract OCR text lines robustly (prefer parsed.lines when valid; otherwise parse raw_response)
- Concatenate into a single continuous text per book (still preserving paragraph breaks)


In [40]:
importlib.reload(MonReader_tools)

from MonReader_tools import (
    coerce_lines, extract_first_json_obj, extract_lines_from_page_obj, join_lines_for_page, extract_book_pages, concatenate_book_text
)

In [43]:
# Run Step I.3 for both books

book_texts = {}

for book_id, _ in BOOKS:
    df_pages = extract_book_pages(book_id)
    display(df_pages[["page","page_n","language","source","n_lines","n_chars"]].head(20))
    print(df_pages["source"].value_counts(dropna=False))

    full_text = concatenate_book_text(df_pages)
    book_texts[book_id] = full_text

    print(f"\n>>> {book_id}: concatenated_chars={len(full_text)} pages={len(df_pages)}")
    print("Preview (first ~1000 chars):")
    print(full_text[:1000])
    print("\n" + "="*80 + "\n")


,page,page_n,language,source,n_lines,n_chars
0,pag0.json,0,guess,raw_fallback,34,1972
1,pag2.json,2,guess,raw_fallback,68,4068
2,pag4.json,4,guess,raw_fallback,66,3992
3,pag6.json,6,guess,raw_fallback,62,3793
4,pag8.json,8,guess,raw_fallback,63,3888
5,pag10.json,10,guess,raw_fallback,69,3712
6,pag12.json,12,guess,raw_fallback,64,4058
7,pag14.json,14,guess,raw_fallback,66,3563
8,pag16.json,16,guess,raw_fallback,65,3513
9,pag18.json,18,guess,raw_fallback,67,3636


source
raw_fallback    12
Name: count, dtype: int64

>>> The_Chamber-John_Grisham: concatenated_chars=43457 pages=12
Preview (first ~1000 chars):
{
  "language": "eng",
  "lines": [
    "Chapter 1 A Delicate Exercise",
    "It began with a phone call on the night of April 17, 1967. Not",
    "trusting his own telephone, Jeremiah Dogan drove to a pay",
    "phone at a gas station to make the call. At the other end, Sam",
    "Cayhall listened to the instructions he was given. When he",
    "returned to bed, he told his wife nothing. She didn’t ask.",
    "Two days later, Cayhall left his home town of Clanton at dusk",
    "and drove to Greenville, Mississippi. There he drove slowly",
    "through the center of the city, and found the offices of the",
    "Jewish lawyer Marvin B. Kramer. It had been easy for the Klan*",
    "to pick Kramer as their next target. He had a long history of",
    "support for the civil rights movement. He led protests against",
    "whites-only facilities. He

,page,page_n,language,source,n_lines,n_chars
0,pag12.json,12,guess,raw_fallback,34,1784
1,pag16.json,16,guess,raw_fallback,71,4409
2,pag18.json,18,guess,raw_fallback,58,3615
3,pag20.json,20,guess,raw_fallback,7,68
4,pag22.json,22,guess,raw_fallback,33,1860
5,pag24.json,24,guess,raw_fallback,70,4331
6,pag26.json,26,guess,raw_fallback,41,4287
7,pag28.json,28,guess,raw_fallback,15,3587
8,pag32.json,32,guess,raw_fallback,71,4351
9,pag36.json,36,guess,raw_fallback,65,4183


source
raw_fallback    12
Name: count, dtype: int64

>>> A_onda_que_se_ergueu_no_mar-Ruy_Castro: concatenated_chars=41033 pages=12
Preview (first ~1000 chars):
{
  "language": "por",
  "lines": [
    "A trilha",
    "sonora de um",
    "país ideal",
    "",
    "Olha que coisa mais linda: as garotas de Ipanema-1961",
    "tomavam cuba-libre, dirigiam Kharman-Ghias e voavam",
    "pela Panair. Usavam frasqueira, vestido-tubinho, cílio",
    "postiço, peruca, laquê. Diziam-se existencialistas, adoravam",
    "arte abstrata e não perdiam um filme da Nouvelle Vague. Seus",
    "points eram o Beco das Garrafas, a Cinemateca, o Arpoador. Iam",
    "à praia com a camisa social do irmão e, sob esta, um biquíni que",
    "de tão insolente, fazia o sangue dos rapazes ferver da maneira",
    "mais inconveniente.",
    "Tudo isso passou. A querida Panair nunca mais voou, a",
    "Nouvelle Vague é um filme em preto e branco e ninguém mais",
    "toma cuba-libre — quem pensaria hoje em misturar rum 

#### Step I.4 — Removal of OCR + formatting artifacts

Goal:
- Remove JSON wrappers, list punctuation, and other non-content artifacts that appear in the concatenated text, while keeping the book text intact.



In [45]:
importlib.reload(MonReader_tools)

from MonReader_tools import (
    remove_json_wrapper_and_list_syntax
)

In [46]:
# Apply to both books
book_texts_clean = {}
for book_id in book_texts:
    cleaned = remove_json_wrapper_and_list_syntax(book_texts[book_id])
    book_texts_clean[book_id] = cleaned

    print(f"\n>>> {book_id}: cleaned_chars={len(cleaned)}")
    print("Preview (first ~600 chars):")
    print(cleaned[:600])
    print("\n" + "="*80)



>>> The_Chamber-John_Grisham: cleaned_chars=37676
Preview (first ~600 chars):
Chapter 1 A Delicate Exercise
It began with a phone call on the night of April 17, 1967. Not
trusting his own telephone, Jeremiah Dogan drove to a pay
phone at a gas station to make the call. At the other end, Sam
Cayhall listened to the instructions he was given. When he
returned to bed, he told his wife nothing. She didn’t ask.
Two days later, Cayhall left his home town of Clanton at dusk
and drove to Greenville, Mississippi. There he drove slowly
through the center of the city, and found the offices of the
Jewish lawyer Marvin B. Kramer. It had been easy for the Klan*
to pick Kramer as thei


>>> A_onda_que_se_ergueu_no_mar-Ruy_Castro: cleaned_chars=36959
Preview (first ~600 chars):
A trilha
sonora de um
país ideal

Olha que coisa mais linda: as garotas de Ipanema-1961
tomavam cuba-libre, dirigiam Kharman-Ghias e voavam
pela Panair. Usavam frasqueira, vestido-tubinho, cílio
postiço, peruca, laquê. Diziam-

#### Step I.5- Repair hyphenated + line-wrapped words

Goal:
- Join hyphenated line breaks: "ex-\nample" -> "example"
- Remove artificial line wraps inside paragraphs, while keeping paragraph breaks



In [47]:
importlib.reload(MonReader_tools)

from MonReader_tools import (
    repair_hyphenation_and_wraps
)

In [48]:
# Apply to both books and preview
book_texts_repaired = {}
for book_id, cleaned in book_texts_clean.items():
    repaired = repair_hyphenation_and_wraps(cleaned)
    book_texts_repaired[book_id] = repaired

    print(f"\n>>> {book_id}: repaired_chars={len(repaired)}")
    print("Preview (first ~600 chars):")
    print(repaired[:600])
    print("\n" + "="*80)


>>> The_Chamber-John_Grisham: repaired_chars=37672
Preview (first ~600 chars):
Chapter 1 A Delicate Exercise It began with a phone call on the night of April 17, 1967. Not trusting his own telephone, Jeremiah Dogan drove to a pay phone at a gas station to make the call. At the other end, Sam Cayhall listened to the instructions he was given. When he returned to bed, he told his wife nothing. She didn’t ask. Two days later, Cayhall left his home town of Clanton at dusk and drove to Greenville, Mississippi. There he drove slowly through the center of the city, and found the offices of the Jewish lawyer Marvin B. Kramer. It had been easy for the Klan* to pick Kramer as thei


>>> A_onda_que_se_ergueu_no_mar-Ruy_Castro: repaired_chars=36765
Preview (first ~600 chars):
A trilha sonora de um país ideal

Olha que coisa mais linda: as garotas de Ipanema-1961 tomavam cuba-libre, dirigiam Kharman-Ghias e voavam pela Panair. Usavam frasqueira, vestido-tubinho, cílio postiço, peruca, laquê. Dizia

#### Step I.6- lock a canonical, continuous prose version

Goal:
Convert the repaired text into a stable, canonical "prose" representation:
- Preserve paragraph breaks as "\n\n"
- Normalize Unicode (NFC)
- Normalize whitespace and punctuation spacing (minimal, no rewriting)


In [54]:
importlib.reload(MonReader_tools)

from MonReader_tools import (
    canonicalize_prose, count_paragraphs
)

In [55]:
book_texts_canonical = {}

for book_id, repaired_text in book_texts_repaired.items():
    canonical = canonicalize_prose(repaired_text)
    book_texts_canonical[book_id] = canonical

    n_paras = count_paragraphs(canonical)
    print(f"\n>>> {book_id}: canonical_chars={len(canonical)} paragraphs≈{n_paras}")
    print("Preview (first ~600 chars):")
    print(canonical[:600])
    print("\n" + "="*80)




>>> The_Chamber-John_Grisham: canonical_chars=37675 paragraphs≈1
Preview (first ~600 chars):
Chapter 1 A Delicate Exercise It began with a phone call on the night of April 17, 1967. Not trusting his own telephone, Jeremiah Dogan drove to a pay phone at a gas station to make the call. At the other end, Sam Cayhall listened to the instructions he was given. When he returned to bed, he told his wife nothing. She didn’t ask. Two days later, Cayhall left his home town of Clanton at dusk and drove to Greenville, Mississippi. There he drove slowly through the center of the city, and found the offices of the Jewish lawyer Marvin B. Kramer. It had been easy for the Klan* to pick Kramer as thei


>>> A_onda_que_se_ergueu_no_mar-Ruy_Castro: canonical_chars=36768 paragraphs≈2
Preview (first ~600 chars):
A trilha sonora de um país ideal

Olha que coisa mais linda: as garotas de Ipanema-1961 tomavam cuba-libre, dirigiam Kharman-Ghias e voavam pela Panair. Usavam frasqueira, vestido-tubinho, cílio p

#### Step I.7 — Sentence-aware chunking (TTS input)

Goal: Turn canonical book text into chunks that:
- Prefer sentence boundaries
- Respect a max char budget (model-agnostic)
- Preserve paragraph boundaries as soft separators
- Are reproducible


In [58]:
importlib.reload(MonReader_tools)

from MonReader_tools import (
    chunk_text_sentence_aware, build_tts_chunks_from_canonical, 
)

In [59]:
# RUN Step I.7 using canonical texts

BOOK_LANG = {
    "The_Chamber-John_Grisham": "eng",
    "A_onda_que_se_ergueu_no_mar-Ruy_Castro": "por",
}

df_chunks = build_tts_chunks_from_canonical(
    book_texts_canonical=book_texts_canonical,
    book_lang=BOOK_LANG,
    max_chars=900
)

print(df_chunks.groupby("book")["chunk_id"].max() + 1)     # chunks per book
display(df_chunks.groupby("book")["n_chars"].agg(["count","mean","max"]))
display(df_chunks.head(8))

# Quick sanity check: should NOT contain JSON artifacts
sample = " ".join(df_chunks["text"].head(3).tolist())
print('Contains JSON artifact \'", "\'? ->', '", "' in sample)
print("Starts with '{' ? ->", sample.lstrip().startswith("{"))


book
A_onda_que_se_ergueu_no_mar-Ruy_Castro    47
The_Chamber-John_Grisham                  44
Name: chunk_id, dtype: int64


,count,mean,max
book,,,
A_onda_que_se_ergueu_no_mar-Ruy_Castro,47,781.319149,897
The_Chamber-John_Grisham,44,855.272727,898


,book,chunk_id,language,n_chars,text
0,The_Chamber-John_Grisham,0,eng,875,Chapter 1 A Delicate Exercise It began with a ...
1,The_Chamber-John_Grisham,1,eng,845,"The operation had been simple to plan, as it i..."
2,The_Chamber-John_Grisham,2,eng,823,"1Highway 61, got in, and drove it out into ope..."
3,The_Chamber-John_Grisham,3,eng,847,The two men climbed into the green Pontiac and...
4,The_Chamber-John_Grisham,4,eng,885,"""Stay by the door and watch the alley, "" Wedge..."
5,The_Chamber-John_Grisham,5,eng,847,"The train passed, and Sam took another wrong t..."
6,The_Chamber-John_Grisham,6,eng,815,""" Sam finally asked, as they turned on to High..."
7,The_Chamber-John_Grisham,7,eng,834,Her husband Marvin helped her to the bathroom ...


Contains JSON artifact '", "'? -> False
Starts with '{' ? -> False


Visualize the entire five chunks:

In [61]:
import textwrap

In [62]:
def pretty_print_chunk(text: str, wrap_width: int = 100):
    """
    Visualization-only formatter:
    - splits on sentence boundaries
    - wraps long sentences for readability
    """
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    for s in sentences:
        if not s:
            continue
        wrapped = textwrap.fill(s, width=wrap_width)
        print(wrapped)
        print()  # blank line between sentences


In [63]:
for i, row in df_chunks.head(5).iterrows():
    print(f"\n=== {row['book']} | chunk_id={row['chunk_id']} | n_chars={row['n_chars']} ===\n")
    pretty_print_chunk(row["text"], wrap_width=90)
    print("\n" + "=" * 80)



=== The_Chamber-John_Grisham | chunk_id=0 | n_chars=875 ===

Chapter 1 A Delicate Exercise It began with a phone call on the night of April 17, 1967.

Not trusting his own telephone, Jeremiah Dogan drove to a pay phone at a gas station to
make the call.

At the other end, Sam Cayhall listened to the instructions he was given.

When he returned to bed, he told his wife nothing.

She didn’t ask.

Two days later, Cayhall left his home town of Clanton at dusk and drove to Greenville,
Mississippi.

There he drove slowly through the center of the city, and found the offices of the Jewish
lawyer Marvin B.

Kramer.

It had been easy for the Klan* to pick Kramer as their next target.

He had a long history of support for the civil rights movement.

He led protests against whites-only facilities.

He accused public officials of racism.

He had paid for the rebuilding of a black church destroyed by the Klan.

He even welcomed Negroes to his home.



=== The_Chamber-John_Grisham | chunk_id=1 | n_

#### Wrap-up and Validation

At the end of **Step I**, we have successfully transformed raw, page-level OCR outputs into a **clean, canonical, and TTS-ready text representation** for both books. The pipeline preserved reading order, semantic continuity, and narrative flow while removing OCR- and formatting-specific artifacts that would interfere with speech synthesis.

The qualitative inspection of the first chunks confirms that:
- The text is **free of JSON syntax, OCR control tokens, and structural noise**.
- Sentence boundaries are preserved, and chunk boundaries respect natural narrative flow.
- Dialogue, punctuation, and paragraph transitions are intact and readable.
- Each chunk falls within a controlled character budget, making it suitable for modern TTS models without truncation or instability.

Crucially, chunking is now performed **exclusively on the canonical continuous prose** derived in Step I.6, ensuring a single, well-defined source of truth for downstream processing. Visualization with line breaks is used only for human inspection and does not alter the actual TTS input.

With these results, **Step I can be considered complete**:
- Inputs to the TTS models are clean, deterministic, and reproducible.
- The text structure aligns with best practices for long-form audiobook-style synthesis.
- The pipeline is robust across languages (English and Portuguese) and book layouts.

This provides a solid and trustworthy foundation to proceed to **Step J — Text-to-Speech Model Benchmarking**, where model quality, prosody, and long-form stability can now be evaluated without confounding data-preparation issues.


----

### Step J – TTS Model Benchmarking  
Evaluate three leading Text-to-Speech models on representative excerpts from both books.  
Comparison dimensions include generation speed, pronunciation accuracy, prosody, multilingual handling (English vs. Portuguese), and long-form stability.